In [3]:
import tekore as tk
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

In [9]:
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [2]:
app_token = tk.request_client_token(CLIENT_ID, CLIENT_SECRET)
tk.Spotify(app_token)

Spotify(token=RefreshingToken(access_token='BQB3ngBxe8-zaJp483sVD659gMGIXfP4lnkbgrZSpWCTn8HzQ6E5i0ar-Ga5STJChejMME9Pu3Rx8F1Al_RTRBNdUDDz5HHOVcIFXx4Yg4qZ5QXw5PA', refresh_token=None, scope=Scope('')), max_limits_on=False, chunked_on=False, sender=SyncSender())

In [21]:
#################
## PREPARATION ##
#################

# Import modules
import sys
# If your authentification script is not in the project directory
# append its folder to sys.path
sys.path.append("../spotify_api_web_app")
# import authorization
import pandas as pd
from tqdm import tqdm
import time

# Authorize and call access object "sp"
sp = tk.Spotify(app_token)

# Get all genres
genres = sp.recommendation_genre_seeds()

In [22]:
len(genres)

126

In [30]:

# Set number of recommendations per genre
n_recs = 50

# Initiate a dictionary with all the information you want to crawl
data_dict = {"id":[], "genre":[], "track_name":[], "artist_name":[],
             "valence":[], "energy":[],"acousticness":[],
             "danceability":[],"liveness":[],"loudness":[],
             "tempo":[],"speechiness":[],"time_signature":[],
             "key":[],"instrumentalness":[]
             }

In [11]:
################
## CRAWL DATA ##
################

# Get recs for every genre
for g in tqdm(genres):
    
    # Get n recommendations
    recs = sp.recommendations(genres = [g], limit = n_recs)
    # json-like string to dict
    recs = eval(recs.json().replace("null", "-999").replace("false", "False").replace("true", "True"))["tracks"]
    
    # Crawl data from each track
    for track in recs:
        try:
            # ID and Genre
            data_dict["id"].append(track["id"])
            data_dict["genre"].append(g)
            # Metadata
            track_meta = sp.track(track["id"])
            data_dict["track_name"].append(track_meta.name)
            data_dict["artist_name"].append(track_meta.album.artists[0].name)

            # music feature
            track_features = sp.track_audio_features(track["id"])
            data_dict["valence"].append(track_features.valence)
            data_dict["energy"].append(track_features.energy)
            data_dict["acousticness"].append(track_features.acousticness)
            data_dict["danceability"].append(track_features.danceability)
            data_dict["liveness"].append(track_features.liveness)
            data_dict["loudness"].append(track_features.loudness)
            data_dict["tempo"].append(track_features.tempo)
            data_dict["speechiness"].append(track_features.speechiness)
            data_dict["time_signature"].append(track_features.time_signature)
            data_dict["key"].append(track_features.key)
            data_dict["instrumentalness"].append(track_features.instrumentalness)
            # Wait 0.2 seconds per track so that the api doesnt overheat
            time.sleep(0.2)
        except Exception as err:
            print(err)
            continue


In [ ]:
##################
## PROCESS DATA ##
##################

# Store data in dataframe
df = pd.DataFrame(data_dict)

# Drop duplicates
df.drop_duplicates(subset = "id", keep = "first", inplace = True)
df.to_csv("valence_arousal_dataset.csv", index = False)